In [1]:
import os
import urllib.request
import urllib.parse
import json
import pandas as pd
import re
from dotenv import load_dotenv

# .env 파일에서 환경 변수 불러오기
load_dotenv()

# 네이버 API 인증 정보 설정
YOUR_CLIENT_ID = os.environ["YOUR_CLIENT_ID"]
YOUR_CLIENT_SECRET = os.environ["YOUR_CLIENT_SECRET"]

# 사용자에게 검색어 입력받기
search_query = input("검색할 키워드를 입력하세요: ")

# 검색어 인코딩
encText = urllib.parse.quote(search_query)

# 네이버 뉴스 검색 API URL
url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&display=100&start=1&sort=sim"

# API 요청
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", YOUR_CLIENT_ID)
request.add_header("X-Naver-Client-Secret", YOUR_CLIENT_SECRET)

# API 응답 처리
response = urllib.request.urlopen(request)
rescode = response.getcode()

def remove_html_tags(html_text):
    """HTML 태그를 제거하는 함수"""
    clean_text = re.sub(r'<[^>]+>', '', html_text)  # 모든 HTML 태그를 제거
    return clean_text

if rescode == 200:
    response_body = response.read()
    data = json.loads(response_body.decode('utf-8'))
    
    NewsData = [
        {'link': item['link']}
        for item in data['items']
        if item['link'].startswith("https://n.news.naver.com/mnews")
    ]

    df_news = pd.DataFrame(NewsData)
else:
    print(f"Error Code: {rescode}")


In [2]:
df_news.head()

,link
0,https://n.news.naver.com/mnews/article/003/001...
1,https://n.news.naver.com/mnews/article/009/000...
2,https://n.news.naver.com/mnews/article/396/000...
3,https://n.news.naver.com/mnews/article/031/000...
4,https://n.news.naver.com/mnews/article/003/001...


In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
import time

def get_news_content(url):
    loader = WebBaseLoader(
        web_paths=[url],
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=["go_trans _article_content"]
            )
        ),
    )
    docs = loader.load()
    time.sleep(1) # 부하 방지
    return docs[0].page_content.strip()

In [4]:
df_news['content'] = df_news['link'].apply(get_news_content) #  apply 사용 하면 병렬 처리
df_news.head()

,link,content
0,https://n.news.naver.com/mnews/article/003/001...,12~14일까지 서울 종로 롤파크서 선발전 진행LCK 서머 우승팀 한화생명과 젠지가...
1,https://n.news.naver.com/mnews/article/009/000...,LCK 대표 선발전한화생명e스포츠와 젠지가 2024 LoL 월드 챔피언십에 LCK ...
2,https://n.news.naver.com/mnews/article/396/000...,2024 LoL 월드 챔피언십 LCK 대표 선발전 대진 및 일정. LCK 제공 ...
3,https://n.news.naver.com/mnews/article/031/000...,12일부터 14일까지 사흘 동안 롤파크에서 선발전 진행라이엇 게임즈의 리그 오브 레...
4,https://n.news.naver.com/mnews/article/003/001...,우리은행이 7일과 8일 경주 실내체육관에서 '2024 LCK 서머 결승전' 관람객을...


In [5]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os

# 환경 변수에서 API 키 가져오기
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# 요약 및 키워드 추출 함수 정의
def summarize_llm(text: str):
    # 프롬프트 템플릿 정의
    prompt_template = """다음 뉴스의 본문의 내용을 7문장 이내로 요약합니다.
    <뉴스 본문> 
    {text}
    </뉴스 본문>

    summary : 
    """

    # 템플릿 생성
    prompt = PromptTemplate.from_template(prompt_template)

    # LLM 생성
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125", api_key=OPENAI_API_KEY)

    # 출력 파서 생성
    output_parser = StrOutputParser()

    # LLM Chain 생성
    llm_chain = prompt | llm | output_parser

    # 입력 텍스트를 chain에 전달하여 요약 및 키워드 추출
    response = llm_chain.invoke({"text": text})

    return response

In [6]:
df_news['summary'] = df_news['content'].apply(summarize_llm) #  apply 사용 하면 병렬 처리
df_news.head()

,link,content,summary
0,https://n.news.naver.com/mnews/article/003/001...,12~14일까지 서울 종로 롤파크서 선발전 진행LCK 서머 우승팀 한화생명과 젠지가...,"서울 종로 롤파크에서 12~14일에 LCK 대표 선발전이 열리며, 한화생명과 젠지가..."
1,https://n.news.naver.com/mnews/article/009/000...,LCK 대표 선발전한화생명e스포츠와 젠지가 2024 LoL 월드 챔피언십에 LCK ...,"한화생명e스포츠와 젠지가 2024 LoL 월드 챔피언십 LCK 대표로 출전 확정, ..."
2,https://n.news.naver.com/mnews/article/396/000...,2024 LoL 월드 챔피언십 LCK 대표 선발전 대진 및 일정. LCK 제공 ...,"2024 LoL 월드 챔피언십 LCK 대표 선발전이 열리고, 한화생명e스포츠와 젠지..."
3,https://n.news.naver.com/mnews/article/031/000...,12일부터 14일까지 사흘 동안 롤파크에서 선발전 진행라이엇 게임즈의 리그 오브 레...,LCK는 '2024 LoL 월드챔피언십' 대표 선발전을 롤파크에서 개최한다. 한화생...
4,https://n.news.naver.com/mnews/article/003/001...,우리은행이 7일과 8일 경주 실내체육관에서 '2024 LCK 서머 결승전' 관람객을...,우리은행이 경주 실내체육관에서 '2024 LCK 서머 결승전' 관람객을 대상으로 이...


In [7]:
df_news.to_csv('naver_api_news.csv', index=False)